In [36]:
import xml.etree.ElementTree as ET
import re
import os.path
import json
from io import BytesIO

#tree = ET.parse(r'C:\Users\Chris\Documents\projects\train-game\drawing.svg')
infile = 'drawing.svg'
svg_outfile = os.path.join('.', 'public', 'ticket-nobg.svg')
react_outfile = os.path.join('.', 'public', 'ticket-nobg.react.svg')
json_outfile = os.path.join('.', 'src', 'app', 'json', 'us_routes.json')

ns = {'svg': 'http://www.w3.org/2000/svg',
      'ttr': 'https://train-game.github.io',
          'inkscape': 'http://www.inkscape.org/namespaces/inkscape'}

ET.register_namespace('', 'http://www.w3.org/2000/svg')
ET.register_namespace('inkscape', 'http://www.inkscape.org/namespaces/inkscape')
ET.register_namespace('sodipodi', 'http://sodipodi.sourceforge.net/DTD/sodipodi-0.dtd')

def fix_doc(node):
    # Remove namespaced attributes from node
    # for attrib_name in list(node.attrib.keys()):
        # if 'inkscape' in attrib_name or 'sodipodi' in attrib_name or 'XML' in attrib_name:
            # del node.attrib[attrib_name]
    for child in node.findall('*'):
        # Strip nodes
        # if 'inkscape' in child.tag or 'sodipodi' in child.tag or child.get('id') == 'BG':
        #     node.remove(child)
        #     continue

        # Rename inkscape:label to id
        label = child.attrib.get('{http://www.inkscape.org/namespaces/inkscape}label')
        if label:
            child.set('id', label)
            # del child.attrib['{http://www.inkscape.org/namespaces/inkscape}label']

    for child in node.findall('*'):
        fix_doc(child)

def make_child_route_ids(root):
    for route in root.find("*/[@id='Routes']").findall('*'):
        # print(route)
        for ix, child in enumerate(route.findall('{http://www.w3.org/2000/svg}g')):
            child.set('id', f"{route.attrib['id']}:{ix}")
            
def make_reacty(node):
    # Make styles react-y (part 1)
    style = node.attrib.get('style')
    if style:
        rules = style.split(';')
        new_rules = []
        for rule in rules:
            key, value = rule.split(':')
            new_key = re.sub(r'(-\w)', lambda result: result.group(1).replace('-', '').upper(), key)
            new_rules.append(f'{new_key}:"{value}"')
        
        node.attrib['style'] = '{{' + ','.join(new_rules) + '}}'
    
    for child in node.findall('*'):
        make_reacty(child)

def to_react_string(svg_node):
    svg_data = ET.tostring(root).decode()
    svg_data = re.sub(r'style="(.*?)"', lambda result: f"style={result.group(1)}", svg_data)
    svg_data = svg_data.replace(r'&quot;', '"')
    return svg_data  # svg_data[:1000]


def add_on_click_to_routes(root):
    for route in root.find("*/[@id='Routes']").findall('*'):
        node.attrib['onClick'] = '{claimRoute}'


def generate_route_json(root):
    route_list = []
    for route in root.find("*/[@id='Routes']").findall('*'):
        children = route.findall('{http://www.w3.org/2000/svg}g')
        if len(children) == 0:
            id = route.attrib['id']
            city1 = id.split('-')[0]
            city2 = id.split('-')[1]
            route_list.append({
                "id": id,
                "city1": city1,
                "city2": city2,
            })
        else:
            for child in children:
                id = child.attrib['id']
                city1 = id.split('-')[0]
                city2 = id.split('-')[1][:-2]
                route_list.append({
                    "id": id,
                    "city1": city1,
                    "city2": city2,
                })
    return json.dumps(route_list)
        


In [37]:
tree = ET.parse(infile)

root = tree.getroot()

fix_doc(root)
make_child_route_ids(root)
# write fixed file
# open(svg_outfile,'w').write(ET.tostring(root).decode())

make_reacty(root)

open(json_outfile, 'w').write(generate_route_json(root))
# add_on_click_to_routes(root)
# open(react_outfile, 'w').write(to_react_string(root))

7436

In [81]:
tree = ET.parse(infile)
root = tree.getroot()
fix_doc(root)
make_child_route_ids(root)
open(f'{1}.{infile}','w').write(ET.tostring(root).decode())

<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF805D4180>
<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF8059B830>
<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF8059BAB0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF80599BC0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF8059B1F0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF8059ADE0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF8059AAC0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF8059B740>
<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF8059BD80>
<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF805BF510>
<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF805BF600>
<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF805BFF10>
<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF805BFB00>
<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF805BF1A0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000001FF805BE1B0>
<Element '{http://www.w3.org/2000/svg}g'

2766133